In [1]:
def split_topics(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    topics = text.split(';\n')
    topics = [topic.strip() for topic in topics if topic.strip()]
    
    return topics

file_path = 'glossario.txt'

topics = split_topics(file_path)

In [2]:
topics[0]

'Abertura de Crédito Adicional\n\n    Ato que torna disponível crédito adicional aprovado por meio de lei ou medida provisória ou ainda por meio de ato infralegal, quando expressamente autorizado em norma com força de lei.\n\n        CF, arts. 166, § 8º, 167, V, e §§ 2º e 3º; Lei nº 4.320/1964, arts. 41 a 44; LDO.\n        Ver também: Crédito Suplementar e Superavit Financeiro .\n        Conceito Específico: Reabertura de Crédito Especial ou Extraordinário .\n        Tradução: Authorization of Supplementary Appropriation (Inglês)'

In [5]:
from langchain_core.documents import Document

def get_documents(topics):
    return [Document(
            page_content=topic,
            metadata={"source": "congressonacional"},
            id=index,
        ) for index,topic in enumerate(topics)]


In [7]:
documents = get_documents(topics)

In [11]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from uuid import uuid4
import os

uuids = [str(uuid4()) for _ in range(len(documents))]
os.environ["OPENAI_API_KEY"] = ""
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
    collection_name="congresso",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

vector_store.add_documents(documents=documents, ids=uuids)

['54ec4573-135e-46b9-ba93-5c97af50a0f5',
 '9c8a293a-f67a-4bee-831d-5bfe3151c129',
 '00a2ee20-dbb6-4ad0-b22f-ecf631d5fb09',
 '8894d055-a52c-43b1-a864-8fbaec000e89',
 'dcc72b18-9bec-4666-afd8-5c406e95c777',
 'ed903f92-60ce-4b6c-a025-c31fa523e34a',
 '2169a073-0fe8-43d1-ba44-7d3ea9a19182',
 '3f1dd014-2e19-44bf-bbef-f54a723bea75',
 '2ce102e7-94cd-46e5-97e2-e08668e9c8f9',
 '3d0af827-e3bf-4de8-86c0-8174099e6599',
 '06cfe212-c8d3-48bb-a916-4d67b6f0042c',
 '722359d9-e396-4479-95dc-4c23159d9d76',
 '4cf2413f-b2e3-463c-aa24-9e1658ba9af2',
 'f688724a-7cae-4f20-8e33-7e416218d324',
 'bf705201-9bea-4711-9a6a-2761fc2acaa0',
 '5d11681d-fd44-4169-9552-6753c4d53a3e',
 '2fd7db0e-5919-4d2b-9802-ec8103341b16',
 'db74198c-0731-4edd-b2cb-be123a127161',
 'd79f8b2f-441c-436e-a7ea-275053b1b39f',
 '1f6a33f7-f7fb-4457-964f-f99289ea249f',
 'e42d8eeb-69d5-4bef-a509-86aa4601dca1',
 '95fa6772-00b8-49ff-bbde-2ad94238e28c',
 'cee416ae-6aee-42af-98a3-ab1bd4e671bf',
 '32803036-f9cc-422b-abe0-f3fd37bcfdb0',
 '5339769d-5c91-

In [17]:
results = vector_store.similarity_search(
    "Abertura de Crédito Adicional ",
    k=2,
    filter={"source": "congressonacional"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Abertura de Crédito Adicional

    Ato que torna disponível crédito adicional aprovado por meio de lei ou medida provisória ou ainda por meio de ato infralegal, quando expressamente autorizado em norma com força de lei.

        CF, arts. 166, § 8º, 167, V, e §§ 2º e 3º; Lei nº 4.320/1964, arts. 41 a 44; LDO.
        Ver também: Crédito Suplementar e Superavit Financeiro .
        Conceito Específico: Reabertura de Crédito Especial ou Extraordinário .
        Tradução: Authorization of Supplementary Appropriation (Inglês) [{'source': 'congressonacional'}]
* Crédito Adicional

    Instrumento de ajuste orçamentário para autorização de despesas não computadas ou insuficientemente dotadas na lei orçamentária.

        Lei nº 4.320/1964, arts. 40 e 41.
        Ver também: Fonte de Recursos para Crédito Adicional , Princípio da Anualidade Orçamentária e Regra de Ouro .
        Conceitos Específicos: Crédito Especial e Crédito Suplementar .
        Tradução: Supplementary Appropriation (In

In [16]:
results

[]

In [6]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
db = SQLDatabase.from_uri("sqlite:///emendas_parlamentares.db")
SQLDatabaseToolkit(db=db, llm=llm).get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [7]:
from agent import GraphAgent

agent = GraphAgent(db,llm)

#agent.invoke("Quais tabelas voce tem acesso?")

In [9]:
agent.tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd201d79510>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [8]:
agent.stream("Quais parlamentares mandaram mais recursos via emendad parlamentares?")

================================ Human Message =================================

Quais parlamentares mandaram mais recursos via emendad parlamentares?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_BL3BthqLDsR6VbTcvu0Wmgfa)
 Call ID: call_BL3BthqLDsR6VbTcvu0Wmgfa
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

ordens_bancarias, tabela_instrumentos
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_6pbbsU16WDAURzwmmsuT4mNk)
 Call ID: call_6pbbsU16WDAURzwmmsuT4mNk
  Args:
    table_names: ordens_bancarias
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE ordens_bancarias (
	ndeg_ob INTEGER, 
	data_ob TIMESTAMP, 
	no_instrumento INTEGER, 
	uf TEXT, 
	municipio TEXT, 
	tipo_de_repasse TEXT, 
	"situacao_(portal_fns)" TEXT, 


'Os parlamentares que mais enviaram recursos via emendas parlamentares, com os respectivos valores totais de repasse, são:\n\n1. **Mara Gabrielli** - R$ 308.459.293,07\n2. **Celso Russomanno** - R$ 260.452.807,74\n3. **Renata Abreu** - R$ 246.732.012,85\n4. **Paulo Paim** - R$ 233.075.642,62\n5. **Jader Barbalho** - R$ 231.429.193,19\n6. **Humberto Costa** - R$ 217.421.214,68\n7. **Pr. Marco Feliciano** - R$ 214.445.423,82\n8. **Tiririca** - R$ 214.379.181,56\n9. **Sérgio Petecão** - R$ 206.030.611,90\n10. **Carlos Zarattini** - R$ 201.628.564,19\n\nEsses valores refletem a soma dos repasses realizados através das emendas parlamentares. Se precisar de mais informações sobre algum parlamentar específico ou sobre o processo de emendas, estou à disposição!'

In [5]:
def print_stream(stream):
    for s in stream:
        last_s = s
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
    if last_s:
        last_message = last_s["messages"][-1]  # Get the last message in the last `s`
        return last_message
a = print_stream(agent.stream("Quais tabelas voce tem acesso?"))

================================ Human Message =================================

Quais tabelas voce tem acesso?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_5CMdAfitBAA5qpV6rpVZ8Vwa)
 Call ID: call_5CMdAfitBAA5qpV6rpVZ8Vwa
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

ordens_bancarias, tabela_instrumentos
================================== Ai Message ==================================

Eu tenho acesso às tabelas "ordens_bancarias" e "tabela_instrumentos". Se você tiver alguma pergunta específica sobre alguma dessas tabelas, por favor, me avise!


In [7]:
a.content

'Eu tenho acesso às tabelas "ordens_bancarias" e "tabela_instrumentos". Se você tiver alguma pergunta específica sobre alguma dessas tabelas, por favor, me avise!'